In [38]:
!git clone https://github.com/Ishikahooda/Tamil-English-Dataset

fatal: destination path 'Tamil-English-Dataset' already exists and is not an empty directory.


In [39]:
import pandas as pd

In [40]:
with open("Tamil-English-Dataset/Dataset/data.en1") as f:
    data_en = f.readlines()
    data_en = data_en[:1500]

with open("Tamil-English-Dataset/Dataset/data.ta1") as f:
    data_ta = f.readlines()
    data_ta = data_ta[:1500]


In [41]:
import fasttext

# Convert the data_ta list to a text file
with open('data_ta.txt', 'w', encoding='utf-8') as f:
    f.writelines(data_ta)

# Train the FastText model
model_ta = fasttext.train_unsupervised('data_ta.txt', model='skipgram')

# Get the word vectors
word_vectors = model_ta.get_input_matrix()
print(len(word_vectors))

# Get the word index
word_index = model_ta.get_words()
print(len(word_index))
# Create a dictionary with the word and the corresponding vector
word_vector_dict = {}
for word, vector in zip(word_index, word_vectors):
    word_vector_dict[word] = vector


2000615
615


In [42]:
len(word_vector_dict)

615

In [43]:
# Convert the data_ta list to a text file
with open('data_en.txt', 'w', encoding='utf-8') as f:
    f.writelines(data_en)

# Train the FastText model
model_en = fasttext.train_unsupervised('data_en.txt', model='skipgram')

# Get the word vectors
word_vectors = model_en.get_input_matrix()

# Get the word index
word_index = model_en.get_words()

In [44]:
from keras.preprocessing.text import Tokenizer
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

def pad_sequences(data, maxlen):
    data = np.array(data)
    data = np.array(list(map(lambda x: x + ['<PAD>'] * (maxlen - len(x)), data)))
    return data

data_ta_split = list(map(lambda x: x.split(), data_ta))
data_ta_pad = pad_sequences(data_ta_split, maxlen=69)

data_en_split = list(map(lambda x: x.split(), data_en))
data_en_pad = pad_sequences(data_en_split, maxlen=69)


def get_embed(model,sentence):
    embed = []
    for word in sentence:
        embed.append(model.get_word_vector(word))

    embed = np.stack(embed, axis=0).astype(np.float32)
    return embed

ta_embed = []
for sentence in data_ta_pad:
    ta_embed += [get_embed(model_ta,sentence)]

en_embed = []
for sentence in data_en_pad:
    en_embed += [get_embed(model_en,sentence)]



ta_embed = np.stack(ta_embed, axis=0).astype(np.float32)
en_embed = np.stack(en_embed, axis=0).astype(np.float32)


c:\Users\Lohith\miniconda3\envs\tensor\lib\site-packages\ipykernel_launcher.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


In [45]:
ta_embed.shape, en_embed.shape

((1500, 69, 100), (1500, 69, 100))

In [46]:


from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, RepeatVector

model = Sequential()
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(128, return_sequences=True))
model.add(Dense(100, activation='elu'))
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
model.build(input_shape=(None, 69, 100))
model.summary()

model.fit(ta_embed, en_embed, epochs=30, batch_size=64)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_8 (LSTM)               (None, 69, 128)           117248    
                                                                 
 lstm_9 (LSTM)               (None, 69, 128)           131584    
                                                                 
 lstm_10 (LSTM)              (None, 69, 128)           131584    
                                                                 
 lstm_11 (LSTM)              (None, 69, 128)           131584    
                                                                 
 dense_2 (Dense)             (None, 69, 100)           12900     
                                                                 
Total params: 524,900
Trainable params: 524,900
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
24/24 [==============================

In [47]:
# Get the sample input from data_ta
sample_input = data_ta[10]
sample_input = sample_input.split()

# Get the word vectors for the sample input
sample_input_embed = get_embed(model_ta, sample_input)

# Get the prediction
prediction = model.predict(np.expand_dims(sample_input_embed, axis=0))

def get_word(vector, model):
    vectors = [model.get_word_vector(word) for word in model.get_words()]
    vectors = np.stack(vectors, axis=0).astype(np.float32)

    # Calculate the cosine similarity
    similarity = vectors @ vector / (np.linalg.norm(vectors, axis=1) * np.linalg.norm(vector))


    # Get the index of the most similar word
    most_similar_index = np.argmax(similarity)

    # Get the word
    words = model_en.get_words()
    most_similar_word = words[most_similar_index]

    return most_similar_word

# Get the predicted sentence
predicted_sentence = []
for word in prediction[0]:
    predicted_sentence.append(get_word(word, model_en))

predicted_sentence = ' '.join(predicted_sentence)

print(f"Input: {' '.join(sample_input)}")
print(f"Predicted: {predicted_sentence}")


1/1 [==============================] - 1s 1s/step
Input: இருவித ஊதிய விகிதங்களும் ஏற்கப்பட்டன மேற்கு ஐரோப்பிய துறைமுகப் பணியாளர்களுக்கும் கிழக்கு ஐரோப்பிய நாடுகள் , ஆசியா இவற்றின் தொழிலாளர்களுக்கும் இடையே பிளவுகளைக் கொண்டுவரும் துறைமுகத் தொழிற்சங்கங்களின் கொள்கை இதே பேரழிவு விளைவுகளைத்தான் கொண்டுவரும் .
Predicted: further children flesh destroy destroy without destroy destroy offering without destroy destroy destroy destroy destroy destroy destroy destroy destroy destroy destroy destroy destroy offering destroy destroy


In [48]:
print(predicted_sentence)

further children flesh destroy destroy without destroy destroy offering without destroy destroy destroy destroy destroy destroy destroy destroy destroy destroy destroy destroy destroy offering destroy destroy
